In [1]:
import pandas as pd 
import numpy as np 
from dowhy import CausalModel 
import warnings
import statsmodels.formula.api as smf

warnings.filterwarnings('ignore')
from tqdm import tqdm

/Users/sawal/opt/anaconda3/envs/causal_ml/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Replication of IHDP datasets

In [27]:
def replicate_ihdp(data_loc, data_name):

    df = pd.read_csv(data_loc)
    causes = ["x" + str(i) for i in range(1, 26)] 
    model = CausalModel(data=df, treatment="treatment", outcome="y", common_causes=causes)
    identified_estimand = model.identify_effect(proceed_when_unidentifiable=True)
    ate = model.estimate_effect(identified_estimand, target_units="ate",
                                     method_name="backdoor.propensity_score_weighting", 
                                     method_params={"weighting_scheme":"ips_weight"}) 
    att = model.estimate_effect(identified_estimand, target_units="att",
                                     method_name="backdoor.propensity_score_matching") 
    atc = model.estimate_effect(identified_estimand, target_units="atc",
                                     method_name="backdoor.propensity_score_matching") 
    diff_in_means = np.mean(df[df["treatment"] == 1]["y"]) - np.mean(df[df["treatment"] == 0]["y"])
    ols = smf.ols(formula="y ~ treatment + {}".format("+".join(causes)), data=df).fit()
    print(data_name)
    print("ATE: {:.3f}, sd: {:.3f}\nATT: {:.3f}, sd: {:.3f}\nATC: {:.3f}, sd: {:.3f}".format(
       ate.value, ate.get_standard_error(), att.value, att.get_standard_error(), 
        atc.value, atc.get_standard_error()))
    print("Diff in means: ", diff_in_means)
    print("OLS estimate:", ols.params["treatment"], "std:", ols.bse["treatment"])
    print("------------------------------------------------------------------------")

In [73]:
def replicate_jobs(data_loc, data_name):

    df = pd.read_csv(data_loc)
    causes = ["x" + str(i) for i in range(0, 17)] 
    model = CausalModel(data=df, treatment="t", outcome="y", common_causes=causes)
    identified_estimand = model.identify_effect(proceed_when_unidentifiable=True)
    ate = model.estimate_effect(identified_estimand, target_units="ate",
                                     method_name="backdoor.propensity_score_weighting", 
                                     method_params={"weighting_scheme":"ips_weight"}) 
    att = model.estimate_effect(identified_estimand, target_units="att",
                                     method_name="backdoor.propensity_score_matching") 
    att2 = model.estimate_effect(identified_estimand, target_units="att", 
                                 method_name="backdoor.distance_matching")
    diff_in_means = np.mean(df[df["t"] == 1]["y"]) - np.mean(df[df["t"] == 0]["y"])
    ols = smf.ols(formula="y ~ t + {}".format("+".join(causes)), data=df).fit()

    print(data_name)
    print("ATE: {:.3f}, sd: {:.3f}\nATT: {:.3f}, sd: {:.3f}".format(
        ate.value, ate.get_standard_error(), att.value, att.get_standard_error()))
    print("Diff in means: ", diff_in_means)
    print("OLS estimate:", ols.params["t"], "std:", ols.bse["t"])
    print("Distance matching: {:.3f}".format(att2.value))
    print("-------------------------------------------------------")

In [74]:
for i in tqdm(range(10)):
    replicate_ihdp("../data/all_data/ihdp_{}.csv".format(i), "ihdp {}".format(i))

  0%|                                                    | 0/10 [00:00<?, ?it/s]


NameError: name 'replicate_ihdp' is not defined

## Replication of Jobs data 

In [75]:
df = pd.read_csv("../data/all_data/jobs_1.csv")
causes = ["x" + str(i) for i in range(17)]
model = CausalModel(data=df, treatment="t", outcome="y", common_causes=causes)
identified_estimand = model.identify_effect(proceed_when_unidentifiable=True)
att = model.estimate_effect(identified_estimand, target_units="att",
                                     method_name="backdoor.propensity_score_matching", 
                                    method_params={"weighting_scheme":"ips_weight"})
print("ATT: {:.3f}, se: {:.3f}".format(att.value, att.get_standard_error()))


ATT: 0.021, se: 0.051


In [76]:
## takes a while to run (about 20 mins)
for i in tqdm(range(10)):
    replicate_jobs("../data/all_data/jobs_{}.csv".format(i), "jobs {}".format(i))

  0%|                                                    | 0/10 [00:00<?, ?it/s]

jobs 0


 10%|████▍                                       | 1/10 [00:19<02:52, 19.18s/it]

ATE: -0.034, sd: 0.032
ATT: 0.059, sd: 0.054
Diff in means:  -0.08897111883976194
OLS estimate: 0.06446299571025646 std: 0.023849629723979464
Distance matching: 0.025
-------------------------------------------------------
jobs 1


 20%|████████▊                                   | 2/10 [00:38<02:34, 19.31s/it]

...

ATE: -0.023, sd: 0.031
ATT: 0.102, sd: 0.052
Diff in means:  -0.08066460430191857
OLS estimate: 0.05943110062672936 std: 0.023680872752272014
Distance matching: 0.047
-------------------------------------------------------
jobs 8


 90%|███████████████████████████████████████▌    | 9/10 [02:56<00:19, 19.73s/it]

ATE: -0.016, sd: 0.030
ATT: 0.138, sd: 0.052
Diff in means:  -0.07479867850505884
OLS estimate: 0.08327435528481993 std: 0.023991410558805264
Distance matching: 0.069
-------------------------------------------------------
jobs 9


100%|███████████████████████████████████████████| 10/10 [03:15<00:00, 19.59s/it]

ATE: -0.030, sd: 0.032
ATT: 0.150, sd: 0.051
Diff in means:  -0.09014105201292977
OLS estimate: 0.06241150192581665 std: 0.02384904138879678
Distance matching: 0.081
-------------------------------------------------------


## Replication of wage 

In [48]:
df = pd.read_csv("../data/all_data/wage.csv")
df["hwage"] = df["wage"]/df["hours"]
df["passed_12"] = df["educ"] > 12
model_1 = smf.ols('np.log(hwage) ~ educ', data=df.assign()).fit()
print(model_1.summary().tables[1])

controls = ['IQ', 'exper', 'tenure', 'age', 'married', 'black',
            'south', 'urban', 'sibs', 'brthord', 'meduc', 'feduc']
model_2 = smf.ols('np.log(hwage) ~ educ +' + '+'.join(controls), data=df).fit()
model_2.summary().tables[1]
model_3 = smf.ols('hwage ~ passed_12', data=df).fit()
print(model_3.summary())

                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      2.2954      0.089     25.754      0.000       2.121       2.470
educ           0.0529      0.007      8.107      0.000       0.040       0.066
                            OLS Regression Results                            
Dep. Variable:                  hwage   R-squared:                       0.062
Model:                            OLS   Adj. R-squared:                  0.061
Method:                 Least Squares   F-statistic:                     61.30
Date:                Tue, 15 Jul 2025   Prob (F-statistic):           1.32e-14
Time:                        23:10:01   Log-Likelihood:                -3437.8
No. Observations:                 935   AIC:                             6880.
Df Residuals:                     933   BIC:                             6889.
Df Model:                           1               

In [ ]:
df['treatment'] = (df['educ'] > 12).astype(int)
model_3 = smf.ols('hwage ~ treatment', data=df).fit()
print(model_3.summary())

## Online classes

In [37]:
data = pd.read_csv("../data/all_data/online_classroom.csv").query("format_blended==0")

result = smf.ols('falsexam ~ format_ol', data=data).fit()
result.summary().tables[1]

,coef,std err,t,P>|t|,[0.025,0.975]
Intercept,78.5475,1.113,70.563,0.000,76.353,80.742
format_ol,-4.9122,1.680,-2.925,0.004,-8.223,-1.601


## Collections email

In [38]:
data = pd.read_csv("../data/all_data/collections_email.csv")
## use FWL theorem
#model_email = smf.ols('email ~ credit_limit + risk_score', data=data).fit()
#model_payments = smf.ols('payments ~ credit_limit + risk_score', data=data).fit()

#residuals = pd.DataFrame(dict(res_payments=model_payments.resid, res_email=model_email.resid))

#model_treatment = smf.ols('res_payments ~ res_email', data=residuals).fit()
#model_treatment.summary().tables[1]

model = smf.ols('payments ~ credit_limit + risk_score + email', data=data).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:               payments   R-squared:                       0.477
Model:                            OLS   Adj. R-squared:                  0.477
Method:                 Least Squares   F-statistic:                     1522.
Date:                Tue, 15 Jul 2025   Prob (F-statistic):               0.00
Time:                        22:36:19   Log-Likelihood:                -28692.
No. Observations:                5000   AIC:                         5.739e+04
Df Residuals:                    4996   BIC:                         5.742e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept      490.8653      9.715     50.527   

## Hospitals

In [66]:
hospital = pd.read_csv("../data/all_data/hospital_treatment.csv")
hosp_4 = smf.ols('days ~ treatment + severity', data=hospital).fit()
print(hosp_4.model.formula)
hosp_4.summary().tables[1]

days ~ treatment + severity


,coef,std err,t,P>|t|,[0.025,0.975]
Intercept,11.6641,2.000,5.832,0.000,7.681,15.647
treatment,-7.5912,2.269,-3.345,0.001,-12.110,-3.073
severity,2.2741,0.154,14.793,0.000,1.968,2.580


## AK91

In [52]:
from linearmodels.iv import IV2SLS

data = pd.read_csv("../data/all_data/ak91.csv")
data['q4'] = (data['quarter_of_birth'] == 4.0).astype(int)
formula = 'log_wage ~ 1 + C(year_of_birth) + C(state_of_birth) + [years_of_schooling ~ q4]'
iv2sls = IV2SLS.from_formula(formula, data).fit()
print(iv2sls.summary)

                          IV-2SLS Estimation Summary                          
Dep. Variable:               log_wage   R-squared:                      0.1217
Estimator:                    IV-2SLS   Adj. R-squared:                 0.1215
No. Observations:              329509   F-statistic:                 1.028e+04
Date:                Tue, Jul 15 2025   P-value (F-stat)                0.0000
Time:                        23:41:47   Distribution:                 chi2(60)
Cov. Estimator:                robust                                         
                                                                              
                                     Parameter Estimates                                     
                           Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
---------------------------------------------------------------------------------------------
Intercept                     4.7468     0.2904     16.348     0.0000      4.1777     

## App engagement

In [53]:
data = pd.read_csv("../data/all_data/app_engagement_push.csv")
iv = IV2SLS.from_formula("in_app_purchase ~ 1 + [push_delivered ~ push_assigned]", data).fit()
iv.summary.tables[1]

,Parameter,Std. Err.,T-stat,P-value,Lower CI,Upper CI
Intercept,69.292,0.3624,191.22,0.0000,68.581,70.002
push_delivered,3.2938,0.7165,4.5974,0.0000,1.8896,4.6981


## Medicine Impact

In [62]:
med = pd.read_csv("../data/all_data/medicine_impact_recovery.csv")
!pip install causalinference
from causalinference import CausalModel
cm = CausalModel(Y=med["recovery"].values, D=med["medication"].values, X=med[["severity", "age", "sex"]].values)
cm.est_via_matching(matches=1, bias_adj=True)
print(cm.estimates)
causes = ["severity", "age", "sex"] 
#model = CausalModel(data=med, treatment="medication", outcome="recovery", common_causes=causes)
#identified_estimand = model.identify_effect(proceed_when_unidentifiable=True)
#ate = model.estimate_effect(identified_estimand, target_units="ate",
#                                     method_name="backdoor.propensity_score_matching") 

#causal_estimate_dmatch = model.estimate_effect(identified_estimand,
#                                              method_name="backdoor.distance_matching",
#                                              target_units="att",
#                                              method_params={'distance_metric':"minkowski", 'p':2})
#print(ate)
#print(causal_estimate_dmatch)


       sex        age  severity  medication  recovery
0        0  35.049134  0.887658           1        31
1        1  41.580323  0.899784           1        49
2        1  28.127491  0.486349           0        38
3        1  36.375033  0.323091           0        35
4        0  25.091717  0.209006           0        15
...    ...        ...       ...         ...       ...
19995    0  58.910288  0.651681           1        40
19996    1  14.383792  0.072034           0         5
19997    0  47.987401  0.431465           0        37
19998    0  27.824466  0.082126           0         6
19999    1  35.185745  0.566556           0        44

[20000 rows x 5 columns]

Treatment Effect Estimates: Matching

                     Est.       S.e.          z      P>|z|      [95% Conf. int.]
--------------------------------------------------------------------------------
           ATE     -7.709      0.609    -12.649      0.000     -8.903     -6.514
           ATC     -6.665      0.246    -27.

## Learning Mindset

In [63]:
data = pd.read_csv("../data/all_data/learning_mindset.csv")
from dowhy import CausalModel 
causes = ["ethnicity", "gender", "school_urbanicity"] 
data_with_categ = pd.concat([data.drop(columns=causes), pd.get_dummies(data[causes], columns=causes, drop_first=False)], axis=1)

common_causes = []
for col in data_with_categ.columns:
    if col != "schoolid" and col != "intervention" and col != "achievement_score":
        common_causes.append(col)
        
model = CausalModel(data=data_with_categ, treatment="intervention", outcome="achievement_score", 
                   common_causes=common_causes)
identified_estimand = model.identify_effect(proceed_when_unidentifiable=True)
ate = model.estimate_effect(identified_estimand, target_units="ate",
                                     method_name="backdoor.propensity_score_weighting", 
                                     method_params={"weighting_scheme":"ips_weight"})
print("ATE: {:.3f}, se: {:.3f}".format(ate.value, ate.get_standard_error()))


ATE: 0.389, se: 0.021


## Billboard impact 

In [64]:
data = pd.read_csv("../data/all_data/billboard_impact.csv")
smf.ols('deposits ~ poa*jul', data=data).fit().summary().tables[1]

,coef,std err,t,P>|t|,[0.025,0.975]
Intercept,171.6423,2.363,72.625,0.000,167.009,176.276
poa,-125.6263,4.484,-28.015,0.000,-134.418,-116.835
jul,34.5232,3.036,11.372,0.000,28.571,40.475
poa:jul,6.5246,5.729,1.139,0.255,-4.706,17.755


## Drinking

In [21]:
drinking = pd.read_csv("../data/all_data/drinking.csv")
drinking["running"] = drinking["agecell"] -21
drinking["threshold"] = drinking["running"] > 0
model = smf.wls("all~running*threshold", drinking).fit()
model.summary().tables[1]

,coef,std err,t,P>|t|,[0.025,0.975]
Intercept,93.6184,0.932,100.399,0.000,91.739,95.498
threshold[T.True],7.6627,1.319,5.811,0.000,5.005,10.320
running,0.8270,0.819,1.010,0.318,-0.823,2.477
running:threshold[T.True],-3.6034,1.158,-3.111,0.003,-5.937,-1.269


## Smoking 2

In [27]:
data = pd.read_csv("../data/all_data/smoking2.csv")
#data = (pd.read_csv("../data/all_data/smoking2.csv")[["state", "year", "cigsale", "california", "after_treatment"]]
#        .rename(columns={"california": "treated"})
#        .replace({"state": {3: "california"}}))
did_model = smf.ols("cigsale ~ after_treatment*california", data=data).fit()
print(did_model.summary())
data["act_passed"] = (data["year"] > 1988) * (data["state"] != 3)
model2 = smf.ols("cigsale ~ C(year) + C(state) + act_passed", data=data).fit()
print(model2.summary())

                            OLS Regression Results                            
Dep. Variable:                cigsale   R-squared:                       0.207
Model:                            OLS   Adj. R-squared:                  0.205
Method:                 Least Squares   F-statistic:                     105.1
Date:                Wed, 16 Jul 2025   Prob (F-statistic):           1.99e-60
Time:                        12:29:31   Log-Likelihood:                -5793.3
No. Observations:                1209   AIC:                         1.159e+04
Df Residuals:                    1205   BIC:                         1.161e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                                                 coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------

## Trainee

In [30]:
trainee = pd.read_csv("../data/all_data/trainee_unique_on_age.csv")
unique_on_age = (trainee.query("trainees==0").drop_duplicates("age"))

from dowhy import CausalModel 
cm = CausalModel(data=trainee, treatment="trainees", outcome="earnings", 
                 common_causes = ["age"])
estimand = cm.identify_effect(proceed_when_unidentifiable=True)
att = cm.estimate_effect(estimand, target_units="att",
                                     method_name="backdoor.propensity_score_matching")
print("att: {:.3f}, se: {:.3f}".format(att.value, att.get_standard_error()))

att: 2457.895, se: 2878.881


## MPs

In [46]:
MPs = pd.read_csv("../data/all_data/MPs.csv")
print(MPs.columns)
# Subset by party
labour = MPs[MPs['party'] == 'labour']
tory = MPs[MPs['party'] == 'tory']

# Tory regressions
tory_fit1 = smf.ols('net ~ margin', data=tory[tory['margin'] < 0]).fit()
tory_fit2 = smf.ols('net ~ margin', data=tory[tory['margin'] > 0]).fit()

# Prediction ranges
y1t_range = [tory['margin'].min(), 0]
y2t_range = [0, tory['margin'].max()]

# Predictions
y1_tory = tory_fit1.predict(pd.DataFrame({'margin': y1t_range}))
y2_tory = tory_fit2.predict(pd.DataFrame({'margin': y2t_range}))


tory_MP = y2_tory.iloc[0]
tory_nonMP = y1_tory.iloc[0]

print("Tory MP avg net wealth:", tory_MP)
print("Tory non-MP avg net wealth:", tory_nonMP)
print(tory_MP - tory_nonMP)

Index(['surname', 'firstname', 'party', 'gross', 'net', 'yob', 'yod',
       'margin.pre', 'region', 'margin'],
      dtype='object')
Tory MP avg net wealth: 13.187801747016026
Tory non-MP avg net wealth: 11.812966722503244
1.374835024512782


In [47]:
data = pd.read_csv("../data/all_data/women.csv")

reg = smf.ols("water ~ reserved", data=data).fit()
print(reg.summary())

                            OLS Regression Results                            
Dep. Variable:                  water   R-squared:                       0.017
Model:                            OLS   Adj. R-squared:                  0.014
Method:                 Least Squares   F-statistic:                     5.493
Date:                Wed, 16 Jul 2025   Prob (F-statistic):             0.0197
Time:                        12:59:32   Log-Likelihood:                -1586.1
No. Observations:                 322   AIC:                             3176.
Df Residuals:                     320   BIC:                             3184.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     14.7383      2.286      6.446      0.0

## Voter Turnout 

In [48]:
data = pd.read_csv("../data/all_data/social.csv")
reg1 = smf.ols("primary2006 ~ primary2004 * C(messages, Treatment(reference='Control'))", data=data).fit()
## see the coefficient for primary2004:C(messages, Treatment(reference='Control'))[T.Neighbors] 
print(reg1.summary())

                            OLS Regression Results                            
Dep. Variable:            primary2006   R-squared:                       0.029
Model:                            OLS   Adj. R-squared:                  0.029
Method:                 Least Squares   F-statistic:                     1326.
Date:                Wed, 16 Jul 2025   Prob (F-statistic):               0.00
Time:                        13:07:50   Log-Likelihood:            -1.9418e+05
No. Observations:              305866   AIC:                         3.884e+05
Df Residuals:                  305858   BIC:                         3.885e+05
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                                                                            coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------

In [60]:
data.to_csv("voter_turnout_new.csv", index=False)
data_small = data[(data["messages"] == "Neighbors") | (data["messages"] == "Control")]
reg1 = smf.ols("primary2006 ~  C(messages, Treatment(reference='Control')) * C(age)", 
               data=data_small).fit()


In [61]:
print(reg1.summary())

                            OLS Regression Results                            
Dep. Variable:            primary2006   R-squared:                       0.028
Model:                            OLS   Adj. R-squared:                  0.027
Method:                 Least Squares   F-statistic:                     39.45
Date:                Wed, 16 Jul 2025   Prob (F-statistic):               0.00
Time:                        13:46:32   Log-Likelihood:            -1.4542e+05
No. Observations:              229444   AIC:                         2.912e+05
Df Residuals:                  229276   BIC:                         2.929e+05
Df Model:                         167                                         
Covariance Type:            nonrobust                                         
                                                                             coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------